In [1]:
import requests
from bs4 import BeautifulSoup

stem_category_url = "https://www.visindavefur.is/vegna_thess.php?id=1"

response = requests.get(stem_category_url)
soup = BeautifulSoup(response.text, "html.parser")

print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="is" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" xmlns:fb="https://www.facebook.com/2008/fbml" xml:lang="is" lang="is">
<![endif]-->
<!--[if IE 7]> <html class="no-js lt-ie9 lt-ie8" lang="is" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" xmlns:fb="https://www.facebook.com/2008/fbml" xml:lang="is" lang="is">
<![endif]-->
<!--[if IE 8]> <html class="no-js lt-ie9" lang="is" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" xmlns:fb="https://www.facebook.com/2008/fbml" xml:lang="is" lang="is"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js lang-is" lang="is" xml:lang="is" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <!--<![endif]-->
 <head>
  <title>
   Vísindavefurinn: Náttúruvísindi og verkfræði
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-s

In [4]:
articles_div = soup.find("div", class_="recent-answers")

articles = []
for article in articles_div.find_all("a", href=True):
    name = article.get_text(strip=True)
    url = article["href"]
    articles.append({"name": name, "url": url})


articles[:2]

[{'name': 'náttúruvísindi og verkfræðiHversu margir bæir eða þorp á Íslandi standa fjarri sjó?',
  'url': '/svar.php?id=70633'},
 {'name': 'náttúruvísindi og verkfræðiHvenær var talan i og tvinntölur uppgötvaðar og til hvers gagnast þær?',
  'url': '/svar.php?id=86796'}]

In [31]:
import re
from bs4 import BeautifulSoup


def extract_main_article_text(soup):
    """
    Extracts the main article text from the given BeautifulSoup object, skipping
    the original question and citations sections.

    Args:
        soup (BeautifulSoup): The BeautifulSoup object containing the parsed HTML.

    Returns:
        str: The extracted article text without the original question or citations.
    """
    # Extract the main article content by selecting the <section> with id 'vv-article-text-area'
    article_section = soup.find("section", {"id": "vv-article-text-area"})

    if article_section:
        # Remove the part containing the original question
        blockquote = article_section.find("blockquote")
        if blockquote:
            # Find all previous siblings of the blockquote and remove them
            previous_sibling = blockquote.find_previous_sibling()
            while previous_sibling:
                previous_sibling.extract()
                previous_sibling = blockquote.find_previous_sibling()
            # Finally, remove the blockquote itself
            blockquote.extract()

        # Remove the citations section (everything after <strong>Tilvísun:</strong>)
        citation = article_section.find("strong", string="Tilvísun:")
        if citation:
            # Get all siblings following the citation and remove them
            next_sibling = citation.find_next_sibling()
            while next_sibling:
                next_sibling.extract()
                next_sibling = citation.find_next_sibling()
            citation.extract()  # Finally, remove the citation itself

        # After removing the citation and original question, get the remaining text
        remaining_text = article_section.get_text(separator="\n").strip()
        normalized_text = re.sub(r"\n{3,}", "\n\n", remaining_text)

        return normalized_text
    else:
        return None


def extract_main_text_naive(soup):
    article_section = soup.find("section", {"id": "vv-article-text-area"})
    return article_section.get_text(separator="\n").strip()


for article in articles:
    article_url = f"https://www.visindavefur.is/{article['url']}"
    article_response = requests.get(article_url)
    article_soup = BeautifulSoup(article_response.text, "html.parser")
    main_text = extract_main_text_naive(article_soup)

    article["main_text"] = main_text

In [45]:
for article in articles:
    article["main_text"] = re.sub(r"\n{3,}", "\n\n", article["main_text"])
    article["main_text"] = re.sub(r"\t{2,}", "\t", article["main_text"])
    article["main_text"] = article["main_text"].replace("\r", "")

articles

[{'name': 'náttúruvísindi og verkfræðiHversu margir bæir eða þorp á Íslandi standa fjarri sjó?',
  'url': '/svar.php?id=70633',
  'main_text': 'Hlusta\n\n\tUpprunalega spurningin hljóðaði svona: \n\n  Hvað eru margir bæir/þorp á Íslandi sem eru ekki við sjó? \n\n\nAð baki orðunum bær og þorp liggja ekki skýrar skilgreiningar, það er ekkert sem segir hvenær húsaþyrping verður að þorpi eða bæ. Hér þarf því að byrja á að ákveða hvaða merkingu á að leggja í orðin. Ein leið er sú að gera ráð fyrir að bær eða þorp hafi sömu merkingu á þéttbýli, líkt og gert er í svari við spurningunni \nHver er fámennasti bær/þorp á Íslandi og hversu margir búa þar?\n Gallinn við það er hins vegar sá að þéttbýli er skilgreint með að lágmarki 200 íbúa sem útilokar þá nokkur vel þekkt þorp. Til þess að ákveða hvaða staði á að taka með í þessu svari er því frekar valið að nota lista Hagstofunnar yfir byggðakjarna með að minnsta kosti 50 íbúa þann 1. janúar 2024.\n[\n1\n]\n\n\nSamkvæmt vef Hagstofunnar eru alls 

In [47]:
import json

with open("../data/visindavefur_articles.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False)